In [1]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline 

from ipynb_path import *

In [2]:
from counterfactual.import_essentials import *
from counterfactual.train import train_model
from counterfactual.evaluate import load_trained_model, model_cf_gen
from counterfactual.net import AdvCounterfactualModel, BaselineModel, CounterfactualModel
from counterfactual.training_module import BaseModule, CounterfactualTrainingModule
from counterfactual.adversarial_experiment import experiment_step, avg_decrease, ExperimentLoggerWanb
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler


SyntaxError: EOL while scanning string literal (adversarial_experiment.py, line 435)

In [10]:
def bn_func(x1, x2, x3, x4):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    return sigmoid(10.5 * ((x1 * x2) / 8100) + 10 - np.random.normal(1, 0.1, 10000) * x3 + 1e-3 * x4)

def x1_to_x3(x1):
    return 1/3 * x1

def x1x2_to_x4(x1, x2):
    return x1 * np.log(x2 **2)

def bn_gen():
    """
    modify code from: https://github.com/divyat09/cf-feasibility/blob/master/generativecf/scripts/simple-bn-gen.py
    """
    x1 = np.random.normal(5, 1, 10000)
    x2 = np.random.normal(-2, 0.7, 10000)
    x3 = x1_to_x3(x1) + np.random.normal(0, 1, 10000)
    x4 = x1x2_to_x4(x1, x2) + np.random.normal(0, 1, 10000)
    y= bn_func(x1, x2, x3, x4)

    data = np.zeros((x1.shape[0], 5))
    data[:, 0] = x1
    data[:, 1] = x2
    data[:, 2] = x3
    data[:, 3] = x4
    data[:, 4] = np.array(y > .5, dtype=np.int)

    return pd.DataFrame(data, columns=['x1', 'x2', 'x3', 'x4', 'y'])

In [4]:
def gen_simulated_data(p: float, n_sample: int):
    sigmoid = lambda x: 1 / (1 + np.exp(-x))
    Z = np.random.binomial(1, p, n_sample)
    x1 = (1 - 2 * Z) * np.random.uniform(0, 1, n_sample)
    x2 = np.random.normal(10, 3, n_sample)
    x3 = x1 * np.log(x2**2)
    y = sigmoid(x1 + x3**2 / 100 - np.array(x2 > 10, dtype=np.int) 
        + 0.1 * np.array(x1>0, dtype=np.int) * np.random.normal(0, 1, n_sample))
    label = np.array(y > 0.5, dtype=np.int)
    data = np.stack((x1, x2, x3, label), axis=1)
    return pd.DataFrame(data, columns=['x1', 'x2', 'x3', 'y'])

In [5]:
m_config = {
    "data_name": "simulated",
    "lr": 0.03,
    "batch_size": 256,
    "lambda_1": 1.0,
    "lambda_2": 0.1,
    "lambda_3": 0.2,
    "threshold": 1.0,
    "continous_cols": [
        "x1", "x2", "x3"
    ],
    "discret_cols": [
    ],
    "enc_dims": [
        3,
        100,
        10
    ],
    "dec_dims": [
        10,
        10
    ],
    "exp_dims": [
        10,
        10
    ],
    "loss_func_1": "mse",
    "loss_func_2": "mse",
    "loss_func_3": "mse",
    "adv_loss_func": "mse"
}
t_config = {
    "max_epochs": 50,
    "gpus": 0,
    "deterministic": True,
    "benchmark": True,
    "automatic_optimization": False
}

In [6]:
gen_simulated_data(0.15, 10000).to_csv("assets/data/adv/simulated/p=0.15.csv")
gen_simulated_data(0.25, 10000).to_csv("assets/data/adv/simulated/p=0.25.csv")

In [248]:
data_dir_list = ['assets/data/adv/simulated/p=0.15.csv', 'assets/data/adv/simulated/p=0.25.csv']
m_config['adv'] = True

m_config['adv_loss_func'] = 'mse'
m_config['eps_scheduler'] = "linear"
results = experiment_step(
    m_config,
    t_config,
    data_dir_list,
    AdvCounterfactualModel,
    return_best_model=False,
    tb_logger_dir=f"simulated",
    experiment_logger=None
)

aggregating data...
total data length: 20000
preprocessing continuous features...
start training...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
hyper parameters: "adv":            True
"adv_loss_func":  mse
"adv_val_models": None
"batch_size":     256
"continous_cols": ['x1', 'x2', 'x3']
"data_dir":       assets/data/adv/simulated/p=0.15.csv
"data_name":      simulated
"dec_dims":       [10, 10]
"discret_cols":   []
"enc_dims":       [3, 100, 10]
"eps_scheduler":  linear
"exp_dims":       [10, 10]
"lambda_1":       1.0
"lambda_2":       0.1
"lambda_3":       0.2
"loss_func_1":    mse
"loss_func_2":    mse
"loss_func_3":    mse
"lr":             0.003
"normalizer":     MinMaxScaler()
"threshold":      1.0
x_cont: (10000, 3), x_cat: (10000, 0)
(10000, 3)

  | Name      | Type                 | Params | In sizes | Out sizes
--------------------------------------------------------------------------
0 | model     | _CounterfactualModel | 1.8 K  | ?        | ?  

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

extracting useful cf results...
calculating validity matrix...


In [7]:
data_dir_list = ['assets/data/adv/simulated/p=0.15.csv', 'assets/data/adv/simulated/p=0.25.csv']
m_config['adv'] = False

m_config['adv_loss_func'] = 'mse'
m_config['eps_scheduler'] = "linear"
results = experiment_step(
    m_config,
    t_config,
    data_dir_list,
    CounterfactualModel,
    return_best_model=False,
    tb_logger_dir=f"simulated",
    experiment_logger=None
)

NameError: name 'experiment_step' is not defined

In [1]:
results['model_list']

NameError: name 'results' is not defined

In [257]:
results['validity_matrix']

[[0.8944000005722046, 0.8636000156402588],
 [0.7900000214576721, 0.6407999992370605]]

In [215]:
from sklearn.linear_model import LogisticRegression

In [223]:
data_1 = gen_simulated_data(0.15, 1000)
clf = LogisticRegression()
clf.fit(data_1[['x1', 'x2', 'x3']], data_1['y'])
clf.score(data_1[['x1', 'x2', 'x3']], data_1['y'])

0.857

In [224]:
data_2 = gen_simulated_data(0.25, 1000)
clf_ = LogisticRegression()
clf_.fit(data_2[['x1', 'x2', 'x3']], data_2['y'])
clf_.score(data_2[['x1', 'x2', 'x3']], data_2['y'])

0.855

In [225]:
clf.coef_

array([[ 1.41513591, -0.92190097,  0.93344707]])

In [226]:
clf_.coef_

array([[ 2.14971431, -0.86389801,  0.68834687]])

In [227]:
clf_.score(data_1[['x1', 'x2', 'x3']], data_1['y'])

0.854